In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.display import FileLink
from tqdm import tqdm_notebook 
from tqdm.notebook import tqdm
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [30]:
train_file = '../input/df-exploded/result_{}.csv'
# super_train = '../input/df-exploded/super_train.csv'

In [31]:
stock = 1
df = pd.read_csv(train_file.format(stock), index_col='ID', parse_dates=['Date'])
df

,stock,unpredictability_score,Close,High,Low,Open,holiday,Date,Open_upper__benford_correlation,Open_hat__benford_correlation,...,Close_trend__benford_correlation,Open_trend__benford_correlation,Open_ trend_upper__benford_correlation,Close_ trend_upper__benford_correlation,Close_trend_lower__benford_correlation,High_lower__benford_correlation,High_trend__benford_correlation,High_upper__benford_correlation,High_trend_lower__benford_correlation,High_ trend_upper__benford_correlation
ID,,,,,,,,,,,,,,,,,,,,,
id_754,1,0,38.4540,38.2343,38.6799,38.4097,1,2017-01-03,0.062915,0.062915,...,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915
id_755,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-04,0.062915,0.062915,...,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915
id_756,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-05,0.062915,0.062915,...,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915
id_757,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-06,0.062915,0.062915,...,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915
id_758,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-09,0.062915,0.062915,...,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_1503,1,0,NaN,NaN,NaN,NaN,0,2019-12-24,-0.064614,-0.064614,...,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614
id_1504,1,0,NaN,NaN,NaN,NaN,1,2019-12-26,-0.064614,-0.064614,...,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614
id_1505,1,0,NaN,NaN,NaN,NaN,0,2019-12-27,-0.064614,-0.064614,...,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614


In [32]:
# tdf = pd.read_csv(super_train, index_col='ID')

# tdf

In [33]:
# print([i for i in df.columns if 'Open' == i])
# print([i for i in df.columns if 'High' == i])
# print([i for i in df.columns if 'Low' == i])
# print([i for i in df.columns if 'holiday' == i])





In [34]:
# df = df.join(tdf[['Open', 'High', 'Low', 'holiday']])

In [35]:
def expand_df(dframe):
    dFrame = dframe.copy()
    dFrame['day'] = dFrame.Date.apply(lambda x: x.day)
    dFrame['month'] = dFrame.Date.apply(lambda x: x.month)
    dFrame['year'] = dFrame.Date.apply(lambda x: x.year)
    dFrame['dayofweek'] = dFrame.Date.apply(lambda x: x.dayofweek)
    dFrame['dayofyear'] = dFrame.Date.apply(lambda x: x.dayofyear)
    dFrame['weekofyear'] = dFrame.Date.apply(lambda x: x.weekofyear)
    dFrame['year_diff'] = dFrame.Date.apply(lambda x: x.year - 2017)
    return dFrame

In [36]:
# df = expand_df(df)

In [37]:
# for col in df.columns:
#     if df[col].isna().sum() != 0: print(col)

In [38]:
df.columns.tolist()

['stock',
 'unpredictability_score',
 'Close',
 'High',
 'Low',
 'Open',
 'holiday',
 'Date',
 'Open_upper__benford_correlation',
 'Open_hat__benford_correlation',
 'Low_hat__benford_correlation',
 'Low_trend__benford_correlation',
 'Low_trend_lower__benford_correlation',
 'Low_ trend_upper__benford_correlation',
 'Close_upper__benford_correlation',
 'Open_hat__fft_coefficient__attr_"abs"__coeff_0',
 'Open_hat__fft_coefficient__attr_"real"__coeff_0',
 'Open_hat__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)',
 'Open_hat__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)',
 'Open_hat__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)',
 'Open_hat__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)',
 'Open_hat__sum_values',
 'Open_hat__quantile__q_0.8',
 'Open_hat__abs_energy',
 'Open_hat__median',
 'Open_hat__mean',
 'Open_hat__quantile__q_0.9',
 'Open_hat__minimum',
 'Open_hat__maximum',
 'Open_hat__quantile__q_0.2',
 'Open_hat__quantile__q_0.3',
 'Open_hat__quant

In [39]:
cat_cols = [ 'holiday','stock',
 'day',
 'month',
 'year',
 'dayofweek',
 'dayofyear',
 'weekofyear',
           'year_diff', 'unpredictability_score']

In [40]:
# encoder = LabelEncoder()
# for col in tqdm(cat_cols):
#     df[col] = encoder.fit_transform(df[col])

In [41]:
# df['train'] = df['Close'].apply(lambda x: not pd.isna(x))
# df.columns = [i.replace('{', '_').replace('}', '_') for i in df.columns]
df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]

In [42]:
X, y = df[df['Close'].notna()].drop(columns=['Close'], axis=1), df[['Close', 'stock']][df['Close'].notna()]
X_test, y_test = df[df['Close'].isna()].drop(columns=['Close'], axis=1), df[['Close', 'stock']][df['Close'].isna()]

In [48]:
X

,stock,unpredictability_score,High,Low,Open,holiday,Date,Open_upper__benford_correlation,Open_hat__benford_correlation,Low_hat__benford_correlation,...,High_upper__benford_correlation,High_trend_lower__benford_correlation,High__trend_upper__benford_correlation,day,month,year,dayofweek,dayofyear,weekofyear,year_diff
ID,,,,,,,,,,,,,,,,,,,,,
id_754,0,0,38.2343,38.6799,38.4097,1,2017-01-03,0.062915,0.062915,0.062915,...,0.062915,0.062915,0.062915,2,0,0,1,1,0,0
id_755,0,0,38.3790,38.8260,38.5551,0,2017-01-04,0.062915,0.062915,0.062915,...,0.062915,0.062915,0.062915,3,0,0,2,2,0,0
id_756,0,0,38.3790,38.8260,38.5551,0,2017-01-05,0.062915,0.062915,0.062915,...,0.062915,0.062915,0.062915,4,0,0,3,3,0,0
id_757,0,0,38.3790,38.8260,38.5551,0,2017-01-06,0.062915,0.062915,0.062915,...,0.062915,0.062915,0.062915,5,0,0,4,4,0,0
id_758,0,0,38.3790,38.8260,38.5551,0,2017-01-09,0.062915,0.062915,0.062915,...,0.062915,0.062915,0.062915,8,0,0,0,7,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_1462,0,0,45.6797,46.1308,45.8535,0,2019-10-25,-0.064614,-0.064614,-0.064614,...,-0.064614,-0.064614,-0.064614,24,9,2,4,290,42,2
id_1463,0,0,45.7155,46.1353,45.8976,0,2019-10-28,-0.064614,-0.064614,-0.064614,...,-0.064614,-0.064614,-0.064614,27,9,2,0,293,43,2
id_1464,0,0,45.7316,46.1443,45.9066,0,2019-10-29,-0.064614,-0.064614,-0.064614,...,-0.064614,-0.064614,-0.064614,28,9,2,1,294,43,2


In [44]:
y

,Close,stock
ID,,
id_754,38.4540,0
id_755,38.5994,0
id_756,38.5994,0
id_757,38.5994,0
id_758,38.5994,0
...,...,...
id_1462,45.9002,0
id_1463,45.9425,0
id_1464,45.9515,0


In [49]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['rmse'],
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 4,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 100000,
    "n_estimators": 1000,
    "random_state": 32
}
model_store = [0] * 103
metrics = [0] * 103
# preds = []
# for stock in tqdm(X.stock.unique(), total=103):

for stock in tqdm([1]):
    X_tr, X_val, y_tr, y_val = train_test_split(X.drop(columns=['Date']), y['Close'], train_size=0.80)
    model_store[stock] = lgb.LGBMRegressor(**hyper_params)
    model_store[stock].fit(X_tr, 
                           y_tr,
                           eval_set=[(X_val, y_val)],
                           eval_metric='rmse',
                           early_stopping_rounds=100,
                           verbose=False,
                           )
    metrics[stock] = list(list(model_store[stock].best_score_.values())[0].values())[0]
    print(stock, metrics[stock])
#     preds.append(pd.DataFrame({'ID': X_test[X_test['stock'] == stock].index, 'Close': model_store[stock].predict(X_test[X_test['stock'] == stock].drop(columns=['Date']), num_iteration=model_store[stock].best_iteration_)}))



# pd.concat(preds).to_csv('result.csv', index=False)
# FileLink('result.csv')

1 0.06436083430864054



In [ ]:
# debug

In [50]:
model_store1 = [0] * 103
metrics1 = [0] * 103

preds1 = []
# for stock in tqdm([1]):
for stock in tqdm(range(103), total=103):
    df = pd.read_csv(train_file.format(stock), index_col='ID', parse_dates=['Date'])
    df = expand_df(df)
    encoder = LabelEncoder()
    for col in cat_cols:
        df[col] = encoder.fit_transform(df[col])
        
    df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]
    X, y = df[df['Close'].notna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].notna()]
    X_test, y_test = df[df['Close'].isna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].isna()]
    
    
    X_tr, X_val, y_tr, y_val = train_test_split(X, y['Close'], train_size=0.80, random_state=11568)
    model_store1[stock] = CatBoostRegressor(loss_function='RMSE', depth=2, learning_rate=0.4, iterations=800, 
        random_seed=18, 
        od_type='Iter',
        od_wait=20,
        thread_count=2
    )
    model_store1[stock].fit(
        X_tr, y_tr, use_best_model=True,
        cat_features=cat_cols,
        eval_set=(X_val, y_val),
        verbose=False,  
        plot=False,
    )
    metrics1[stock] = model_store1[stock].best_score_['validation']['RMSE']
    print(stock, metrics1[stock])
    preds1.append(pd.DataFrame({'ID': X_test.index, 'Close': model_store1[stock].predict(X_test)}))
#     pd.concat(preds1).to_csv('result1.csv', index=False)
    

0 0.09665492208084608



In [ ]:
# FileLink('result1.csv')

In [ ]:
# preds = []
# for stock in X.stock.unique(): 

# # for stock in [1]:
#     pred1 = model_store[stock].predict(X_test[X_test['stock'] == stock].drop(columns=['Date']), num_iteration=model_store[stock].best_iteration_)
#     pred2 = model_store1[stock].predict(X_test[X_test['stock'] == stock])
    
#     if metrics[stock] > metrics1[stock]:
#         print("picked first")
# #         preds.append(pd.DataFrame({'ID': X_test[X_test['stock'] == stock].index, 'Close': model_store1[stock].predict(X_test[X_test['stock'] == stock])}))
#         preds.append(pd.DataFrame({'ID': X_test[X_test['stock'] == stock].index, 'Close': (pred2 * 7 + pred1) / 8}))
    
#     else:
#         print("picked second")
# #         preds.append(pd.DataFrame({'ID': X_test[X_test['stock'] == stock].index, 'Close': model_store[stock].predict(X_test[X_test['stock'] == stock].drop(columns=['Date']), num_iteration=model_store[stock].best_iteration_)}))
#         preds.append(pd.DataFrame({'ID': X_test[X_test['stock'] == stock].index, 'Close': (pred2 * 4 + pred1) / 5}))
    

pd.concat(preds1).to_csv('result.csv', index=False)
pd.concat(preds1).shape       

In [ ]:
FileLink('result.csv')

In [ ]:
# for stock in X_test.stock.unique():
!cat result.csv

In [ ]:
# !cat result.csv

In [ ]:
# !cat result1.csv

In [ ]:
# for stock in X_test.stock.unique():
#     print(stock, model_store[stock].feature_importances_[:10])

In [ ]:
# subs = pd.concat(preds).set_index('ID', drop=True)

In [ ]:
# df['Close'].fillna(subs.Close, inplace=True)

In [ ]:
# df[df['stock'] == 41]['High'].plot(style='k')

In [ ]:
# df[df['stock'] == 41]['Close'].plot(style='k')

In [ ]:
# df[df['stock'] == 41]['Low'].plot(style='k')